In [0]:
# 1) Instalação da dependência Kaggle
%pip install kaggle
%pip install kagglehub 

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession
import os
import json
from datetime import datetime
import kagglehub

In [0]:
# 1. Configurar ambiente Kaggle a partir do arquivo no DBFS
def setup_kaggle_credentials():
    try:
        # Leitura do arquivo usando DBFS path
        kaggle_json_path = "/dbfs/FileStore/tables/kaggle.json"
        
        # Verificar se o arquivo existe
        if not os.path.exists(kaggle_json_path):
            print(f"Arquivo não encontrado: {kaggle_json_path}")
            return False
        
        # Carregar credenciais do arquivo
        with open(kaggle_json_path) as f:
            creds = json.load(f)
        
        # Configurar variáveis de ambiente
        os.environ['KAGGLE_USERNAME'] = creds['username']
        os.environ['KAGGLE_KEY'] = creds['key']
        
        print("Credenciais do Kaggle configuradas com sucesso!")
        return True
    except Exception as e:
        print(f"Erro ao configurar credenciais: {str(e)}")
        return False
    

In [0]:
# 2. Configuração inicial do Spark
spark = SparkSession.builder.appName("KaggleToDBFS").getOrCreate()

In [0]:
# 3. Verificar/Criar pasta no DBFS
def ensure_directory_exists(path):
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    dbfs_path = spark._jvm.org.apache.hadoop.fs.Path(path)
    if not fs.exists(dbfs_path):
        print(f"Criando diretório: {path}")
        fs.mkdirs(dbfs_path)
    else:
        print(f"Diretório já existe: {path}")

dbfs_dir = "/FileStore/tables/data_ingestion"
ensure_directory_exists(dbfs_dir)


Diretório já existe: /FileStore/tables/data_ingestion


In [0]:
# 4. Configurar credenciais Kaggle
print("\nConfigurando credenciais Kaggle...")
if not setup_kaggle_credentials():
    # Tentar método alternativo usando dbutils
    print("Tentando método alternativo de leitura...")
    try:
        # Ler o arquivo diretamente do DBFS usando dbutils
        kaggle_json_content = dbutils.fs.head("/FileStore/tables/kaggle.json", 10000)
        creds = json.loads(kaggle_json_content)
        
        os.environ['KAGGLE_USERNAME'] = creds['username']
        os.environ['KAGGLE_KEY'] = creds['key']
        
        print("Credenciais configuradas com sucesso via dbutils!")
    except Exception as alt_e:
        print(f"Falha no método alternativo: {str(alt_e)}")
        dbutils.notebook.exit("Falha na configuração das credenciais. Abortando.")



Configurando credenciais Kaggle...
Arquivo não encontrado: /dbfs/FileStore/tables/kaggle.json
Tentando método alternativo de leitura...
Credenciais configuradas com sucesso via dbutils!


In [0]:
# 5. Download do dataset
print("\n" + "="*50)
print("Iniciando download do dataset...")
try:
    download_path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")
    print(f"Arquivo baixado em: {download_path}")
except Exception as e:
    print(f"Erro no download: {str(e)}")
    raise e


Iniciando download do dataset...


100%|██████████| 1.95M/1.95M [00:00<00:00, 27.2MB/s]

Extracting files...
Arquivo baixado em: /root/.cache/kagglehub/datasets/karkavelrajaj/amazon-sales-dataset/versions/1


In [0]:
# 6. Função para remover arquivos existentes do mesmo dia
def remove_existing_files_for_today():
    # Formato de data para comparação (AAAAMMDD)
    today_str = datetime.now().strftime("%Y%m%d")
    
    # Listar arquivos no diretório
    file_list = dbutils.fs.ls(dbfs_dir)
    
    # Filtrar arquivos que começam com a data de hoje
    for file_info in file_list:
        file_name = os.path.basename(file_info.path)
        if file_name.startswith(today_str + "_"):
            print(f"Removendo arquivo existente do mesmo dia: {file_info.path}")
            dbutils.fs.rm(file_info.path)

# 7. Remover arquivos existentes do mesmo dia antes de copiar
print("\nVerificando arquivos existentes do mesmo dia...")
remove_existing_files_for_today()


Verificando arquivos existentes do mesmo dia...


In [0]:
# 8. Copiar arquivos para DBFS com novo nome (apenas data)
def copy_to_dbfs(src_path):
    # Listar todos os arquivos no diretório de download
    for file_name in os.listdir(src_path):
        file_path = os.path.join(src_path, file_name)
        
        if os.path.isfile(file_path):
            # Gerar novo nome com apenas a data (sem horário)
            date_str = datetime.now().strftime("%Y%m%d")
            new_file_name = f"{date_str}_{file_name}"
            dbfs_dest_path = f"{dbfs_dir}/{new_file_name}"
            
            # Copiar arquivo para DBFS
            dbutils.fs.cp(f"file:{file_path}", dbfs_dest_path)
            print(f"Arquivo copiado: {file_name} -> {dbfs_dest_path}")
            
            return dbfs_dest_path

print("\n" + "="*50)
print("Copiando arquivos para DBFS...")
final_destination = copy_to_dbfs(download_path)


Copiando arquivos para DBFS...
Arquivo copiado: amazon.csv -> /FileStore/tables/data_ingestion/20250707_amazon.csv


In [0]:
# 9. Resultado final
print("\n" + "="*50)
print("Resumo da operação:")
print(f"Dataset baixado de: kagglehub.dataset_download('karkavelrajaj/amazon-sales-dataset')")
print(f"Credenciais carregadas de: /FileStore/tables/kaggle.json")
print(f"Arquivo final salvo em: {final_destination}")
print("="*50)


Resumo da operação:
Dataset baixado de: kagglehub.dataset_download('karkavelrajaj/amazon-sales-dataset')
Credenciais carregadas de: /FileStore/tables/kaggle.json
Arquivo final salvo em: /FileStore/tables/data_ingestion/20250707_amazon.csv


In [0]:
files = dbutils.fs.ls("/FileStore/tables/data_ingestion/")
display(files)

path,name,size,modificationTime
dbfs:/FileStore/tables/data_ingestion/20250616_amazon.csv,20250616_amazon.csv,4744481,1750079913000
dbfs:/FileStore/tables/data_ingestion/20250617_amazon.csv,20250617_amazon.csv,4744481,1750158081000
dbfs:/FileStore/tables/data_ingestion/20250618_amazon.csv,20250618_amazon.csv,4744481,1750244525000
dbfs:/FileStore/tables/data_ingestion/20250707_amazon.csv,20250707_amazon.csv,4744481,1751884362000


In [0]:
#teste
from pyspark.sql import SparkSession

amazon = spark.read.csv("/FileStore/tables/data_ingestion/20250618_amazon.csv", header=True, sep=",",inferSchema=True)
amazon.show(5)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|        product_name|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|            ₹399|      ₹1,099|                64%|   4.2|      2